<a href="https://colab.research.google.com/github/mburkey3/Intro_to_ML/blob/main/Homework6/Homework6_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.optim as optim
import pandas as pd
import torch.nn as nn
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
file_path = '/content/drive/My Drive/Intro_to_ML/Datasets/Housing.csv'
dataset = pd.DataFrame(pd.read_csv(file_path))
dataset.head()

,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus
0,13300000,7420,4,2,3,yes,no,no,no,yes,2,yes,furnished
1,12250000,8960,4,4,4,yes,no,no,no,yes,3,no,furnished
2,12250000,9960,3,2,2,yes,no,yes,no,no,2,yes,semi-furnished
3,12215000,7500,4,2,2,yes,no,yes,no,yes,3,yes,furnished
4,11410000,7420,4,1,2,yes,yes,yes,no,yes,2,no,furnished


In [3]:
# Binary map Outputs to 0 or 1
# M = 1
# B = 0
# Map function
def binary_map(x):
    return x.map({'yes': 1, 'no': 0})

varlist = ['mainroad', 'guestroom', 'basement', 'hotwaterheating', 'airconditioning', 'prefarea']
dataset[varlist] = dataset[varlist].apply(binary_map)
# Remove uneeded information
dataset.pop('furnishingstatus')
dataset.head()

,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea
0,13300000,7420,4,2,3,1,0,0,0,1,2,1
1,12250000,8960,4,4,4,1,0,0,0,1,3,0
2,12250000,9960,3,2,2,1,0,1,0,0,2,1
3,12215000,7500,4,2,2,1,0,1,0,1,3,1
4,11410000,7420,4,1,2,1,1,1,0,1,2,0


In [4]:
# Set up inputs and outputs
Y = dataset.pop('price')
X = dataset.values
# convert to tensors
X = torch.tensor(X, dtype=torch.float32)
Y = torch.tensor(Y, dtype=torch.float32)

In [5]:
# Normalizing the data
X = nn.functional.normalize(X, dim = 1)
Y = nn.functional.normalize(Y, dim = 0)
X.size(), Y.size()

(torch.Size([545, 11]), torch.Size([545]))

In [6]:
# Split data into training and validation sets
n_samples = X.shape[0]
n_val = int(0.2 * n_samples)

shuffled_indices = torch.randperm(n_samples)

train_indices = shuffled_indices[:-n_val]
val_indices = shuffled_indices[-n_val:]

train_indices.size(), val_indices.size()

(torch.Size([436]), torch.Size([109]))

In [7]:
# Training set
train_X = X[train_indices]
train_Y = Y[train_indices]
# Validation set
val_X = X[val_indices]
val_Y = Y[val_indices]

train_X.size()

torch.Size([436, 11])

In [8]:
# Building the model
model = nn.Sequential(
    nn.Linear(11, 32),
    nn.Tanh(),
    nn.Linear(32, 1)
)

In [9]:
# Training Function
def training_fn(epochs, opt, model, loss_fn, x_train, y_train, x_val, y_val) :
    for epoch in range(1, epochs + 1):
        train_p = model(x_train.view(x_train.shape[0], -1))
        train_loss = loss_fn(train_p, y_train)

        val_p = model(x_val)
        val_loss = loss_fn(val_p, y_val)

        opt.zero_grad()
        train_loss.backward()
        opt.step()

        if epoch % 500 == 0:
            print(f"Epoch {epoch}, Training loss {train_loss.item():.4f},"
                  f" Validation loss {val_loss.item():.4f}")

    return

In [10]:
# Training the model
optimizer = optim.SGD(model.parameters(), lr = 0.0001)

training_fn(
    epochs = 5000,
    opt = optimizer,
    model = model,
    loss_fn = nn.MSELoss(),
    x_train = train_X,
    y_train = train_Y,
    x_val = val_X,
    y_val = val_Y
)

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([436])) that is different to the input size (torch.Size([436, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([109])) that is different to the input size (torch.Size([109, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch 500, Training loss 0.0338, Validation loss 0.0334
Epoch 1000, Training loss 0.0159, Validation loss 0.0156
Epoch 1500, Training loss 0.0075, Validation loss 0.0074
Epoch 2000, Training loss 0.0036, Validation loss 0.0035
Epoch 2500, Training loss 0.0018, Validation loss 0.0018
Epoch 3000, Training loss 0.0010, Validation loss 0.0010
Epoch 3500, Training loss 0.0006, Validation loss 0.0006
Epoch 4000, Training loss 0.0004, Validation loss 0.0004
Epoch 4500, Training loss 0.0003, Validation loss 0.0003
Epoch 5000, Training loss 0.0003, Validation loss 0.0003


In [11]:
## Homework 6B
# Training set
train_X = X[train_indices]
train_Y = Y[train_indices]
# Validation set
val_X = X[val_indices]
val_Y = Y[val_indices]

train_X.size()

torch.Size([436, 11])

In [12]:
modelB = nn.Sequential(
    nn.Linear(11, 32),
    nn.Tanh(),
    nn.Linear(32, 64),
    nn.Tanh(),
    nn.Linear(64, 16),
    nn.Tanh(),
    nn.Linear(16, 1)
)

In [13]:
training_fn(
    epochs = 5000,
    opt = optimizer,
    model = modelB,
    loss_fn = nn.MSELoss(),
    x_train = train_X,
    y_train = train_Y,
    x_val = val_X,
    y_val = val_Y
)

Epoch 500, Training loss 0.0399, Validation loss 0.0404
Epoch 1000, Training loss 0.0399, Validation loss 0.0404
Epoch 1500, Training loss 0.0399, Validation loss 0.0404
Epoch 2000, Training loss 0.0399, Validation loss 0.0404
Epoch 2500, Training loss 0.0399, Validation loss 0.0404
Epoch 3000, Training loss 0.0399, Validation loss 0.0404
Epoch 3500, Training loss 0.0399, Validation loss 0.0404
Epoch 4000, Training loss 0.0399, Validation loss 0.0404
Epoch 4500, Training loss 0.0399, Validation loss 0.0404
Epoch 5000, Training loss 0.0399, Validation loss 0.0404
